# CONSTRAINT SATISFACTION PROBLEMS (CSP)

This IPy notebook uses the code and examples provided in the supporting materials of the book *Artificial Intelligence: A Modern Approach*.

In [1]:
from csp import *
# from notebook import psource, plot_NQueens

# %matplotlib inline
# Hide warnings in the matplotlib sections

import math
import warnings
warnings.filterwarnings("ignore")

----
## Zebra puzzle
Solves an instance of the "zebra puzzle".<br>

----

### Approach 01: N-ary CSP
In this approach, constraints can be global (N-ary).

A nary-CSP consists of:
* *variables*   : a set of variables
* *domains*     : a dictionary that maps each variable to its domain
* *constraints* : a list of constraints
* *var_to_const*: a dictionary that maps each variable to its set of constraints

In [2]:
# Solve Zebra using NaryCPS and AC_Sover

# variables
Colors = 'Red Yellow Blue Green Ivory'.split()
Pets = 'Dog Fox Snails Horse Zebra'.split()
Drinks = 'OJ Tea Coffee Milk Water'.split()
Countries = 'Englishman Spaniard Norwegian Ukranian Japanese'.split()
Smokes = 'Kools Chesterfields Winston LuckyStrike Parliaments'.split()
variables = set (Colors + Pets + Drinks + Countries + Smokes)

# Domains
dominio = {}
for var in variables:
    dominio[var] = set(range(1, 6))     # list(range(1, 6))
dominio['Norwegian'] = {1}
dominio['Milk'] = {3}

# Constraints
restricoes = [
              Constraint(Colors, all_diff_constraint),
              Constraint(Pets, all_diff_constraint),
              Constraint(Drinks, all_diff_constraint),
              Constraint(Countries, all_diff_constraint),
              Constraint(Smokes, all_diff_constraint),
              Constraint(('Englishman', 'Red'), eq),
              Constraint(('Spaniard', 'Dog'), eq),
              Constraint(('Kools', 'Yellow'), eq),
              Constraint(('Winston', 'Snails'), eq),
              Constraint(('LuckyStrike', 'OJ'), eq),
              Constraint(('Ukranian', 'Tea'), eq),
              Constraint(('Japanese', 'Parliaments'), eq),
              Constraint(('Coffee', 'Green'), eq),
              Constraint(('Chesterfields', 'Fox'), lambda a, b: abs(a - b) == 1),
              Constraint(('Norwegian', 'Blue'), lambda a, b: abs(a - b) == 1),
              Constraint(('Kools', 'Horse'), lambda a, b: abs(a - b) == 1),
              Constraint(('Green', 'Ivory'), lambda a, b: b == a - 1),
              ]

# Define NaryCSP
zebra_narycsp = NaryCSP(dominio, restricoes)

# Apply solver
sol_nary = ac_solver(zebra_narycsp, arc_heuristic=sat_up)

# Print result
for h in range(1, 6):
        print('House', h, end=' ')
        for (var, val) in sol_nary.items():
            if val == h:
                print(var, end=' ')
        print()

House 1 Water Yellow Norwegian Kools Fox 
House 2 Blue Horse Chesterfields Tea Ukranian 
House 3 Milk Red Snails Winston Englishman 
House 4 Dog LuckyStrike Spaniard Ivory OJ 
House 5 Coffee Japanese Zebra Parliaments Green 


----
### Approach 02: Binary CSP
In this approach, constraints are binary and we need to identify each variable's neighbors.

A CSP is specified by the following inputs:
* *variables*: A list of variables; each is atomic (e.g. int or string).
* *domains*: A dict of {var:[possible_value, ...]} entries.
* *neighbors*: A dict of {var:[var,...]} that for each variable lists the other variables that participate in constraints.
* *constraints*: A function f(A, a, B, b) that returns true if neighbors
                    A, B satisfy the constraint when they have values A=a, B=b
                    
Constraints can be specified as explicit pairs of allowable values.

In [3]:
def zebra_csp():
    # variables
    Colors = 'Red Yellow Blue Green Ivory'.split()
    Pets = 'Dog Fox Snails Horse Zebra'.split()
    Drinks = 'OJ Tea Coffee Milk Water'.split()
    Countries = 'Englishman Spaniard Norwegian Ukranian Japanese'.split()
    Smokes = 'Kools Chesterfields Winston LuckyStrike Parliaments'.split()
    z_variables = Colors + Pets + Drinks + Countries + Smokes

    # Domains
    z_domains = {}
    for var in z_variables:
        z_domains[var] = list(range(1, 6))
    z_domains['Norwegian'] = [1]
    z_domains['Milk'] = [3]

    # Neighbors
    z_neighbors = parse_neighbors("""Englishman: Red;
                Spaniard: Dog; Kools: Yellow; Chesterfields: Fox;
                Norwegian: Blue; Winston: Snails; LuckyStrike: OJ;
                Ukranian: Tea; Japanese: Parliaments; Kools: Horse;
                Coffee: Green; Green: Ivory""")
    # print(z_neighbors)

    # Add the all_diff_constraint for the itens of the same type 
    for type in [Colors, Pets, Drinks, Countries, Smokes]:
            for A in type:
                for B in type:
                    if A != B:
                        if B not in z_neighbors[A]:
                            z_neighbors[A].append(B)
                        if A not in z_neighbors[B]:
                            z_neighbors[B].append(A)
    # print(z_neighbors)

    # Define the constraint function
    def z_constraints(A, a, B, b, recurse=0):
            same = (a == b)
            next_to = abs(a - b) == 1
            if A == 'Englishman' and B == 'Red':
                return same
            if A == 'Spaniard' and B == 'Dog':
                return same
            if A == 'Chesterfields' and B == 'Fox':
                return next_to
            if A == 'Norwegian' and B == 'Blue':
                return next_to
            if A == 'Kools' and B == 'Yellow':
                return same
            if A == 'Winston' and B == 'Snails':
                return same
            if A == 'LuckyStrike' and B == 'OJ':
                return same
            if A == 'Ukranian' and B == 'Tea':
                return same
            if A == 'Japanese' and B == 'Parliaments':
                return same
            if A == 'Kools' and B == 'Horse':
                return next_to
            if A == 'Coffee' and B == 'Green':
                return same
            if A == 'Green' and B == 'Ivory':
                return a - 1 == b
            if recurse == 0:
                return z_constraints(B, b, A, a, 1)
            if ((A in Colors and B in Colors) or
                    (A in Pets and B in Pets) or
                    (A in Drinks and B in Drinks) or
                    (A in Countries and B in Countries) or
                    (A in Smokes and B in Smokes)):
                return not same
            raise Exception('error')

    # Return a zebra CSP
    return CSP(z_variables, z_domains, z_neighbors, z_constraints)

### Solve Zebra using backtracing search

In [8]:
# Solve Zebra using backtracking search

# zebra_bt = zebra_csp()
zebra_bt = zebra_narycsp
sol_bt = backtracking_search (zebra_bt, select_unassigned_variable=mrv, order_domain_values=lcv, inference=forward_checking)
for h in range(1, 6):
    print('House', h, end=' ')
    for (var, val) in sol_bt.items():
        if val == h:
            print(var, end=' ')
    print()
print(sol_bt)
print('\nNs assigments: ', zebra_bt.nassigns)



AttributeError: 'NaryCSP' object has no attribute 'curr_domains'

### Solve Zebra using local search

Zebra is not a good case for the application of local search.

You will get "No result" if the algorithm reachs max_steps with conflicts.
So you have to do several trials!

In [5]:
# ______________________________________________________________________________
# The Zebra Puzzle

zebra_mc = zebra_csp()
sol_mc = min_conflicts(zebra_mc, max_steps=1000)

if zebra_mc.nassigns < 1000:
    print(sol_mc)
    for h in range(1, 6):
        print('House', h, end=' ')
        for (var, val) in sol_mc.items():
            if val == h:
                print(var, end=' ')
        print()
else:
    print("No result.")

print('\nNr. assigments: ', zebra_mc.nassigns)

No result.

Nr. assigments:  1025


----
## cryptarithmetic problem
### **Puzzle D A Y S + T O O = S H O R T**

Each letter stands for a distinct digit; the aim is to find a substitution of digits for letters such that the resulting sum is arithmetically correct, with the added restriction that no leading zeroes are allowed.


### Approach: NaryCSP + AC_Solver
----

In [6]:
# DAYS + TOO = SHORT 
# NARY_CSP AC_Sover

# variables
dts_vars = 'A D H O R S T Y'.split()

# domain
dominio = {
          'D': set(range(1, 10)), 'A': set(range(0, 10)), 'Y': set(range(0, 10)), 'S': set(range(1, 10)),
          'T': set(range(1, 10)), 'O': set(range(0, 10)), 'H': set(range(0, 10)), 'R': set(range(0, 10)),
          'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2)), 'C4': set(range(0, 2))
          }

# constraints
restricoes = [
              Constraint(dts_vars, all_diff_constraint),
              Constraint(('S', 'O', 'T', 'C1'), lambda s, o, t, c1: s + o == t + 10 * c1),
              Constraint(('Y', 'O', 'R', 'C1', 'C2'), lambda y, o, r, c1, c2: c1 + y + o == r + 10 * c2),
              Constraint(('A', 'T', 'O', 'C2', 'C3'), lambda a, t, o, c2, c3: c2 + a + t == o + 10 * c3),
              Constraint(('D', 'H', 'C3', 'C4'), lambda d, h, c3, c4: c3 + d == h + 10 * c4),
              Constraint(('S', 'C4'), eq)
              ]

# days_too_short
days_too_short = NaryCSP(dominio, restricoes)

# print(days_too_short.variables)
# print(days_too_short.domains)

# Result
# ac_search_solver(days_too_short, arc_heuristic=sat_up)
soln_dts = ac_solver(days_too_short, arc_heuristic=sat_up)

# Print result
print('Days:   ',end=' ')
for var in 'D A Y S'.split():
    print(soln_dts[var], end=' ')
print()
print('Too:      ',end=' ')
for var in 'T O O'.split():
    print(soln_dts[var], end=' ')
print('\n       ---------')
print('Short:',end=' ')
for var in 'S H O R T'.split():
    print(soln_dts[var], end=' ')
print()

Days:    9 8 7 1 
Too:       6 5 5 
       ---------
Short: 1 0 5 2 6 


----
## "Get a ride to Campus" Problem

How to assign passengers to vehicles from home/work to Campus and from Campus to home, minimizing the number of required trips.
* Each vehicle owner can give a ride of 1 to 4 passengers;
* The vehicle owners and passengers should be assigned to one single location;
* Passengers from distinct locations in the same path to the Campus can be assigned to the same trip;
* Each passenger has a schedule that defines the latest hour (min) to be on Campus and earliest hour (max) to leave the Campus;
* If we consider the tree of paths from the Campus to the locations, each main branch can be treated independently.

In the case of IPCA, we could consider:
* Locations: Amr, Braga, Gmr, Joane, PL, Prado, PV, Trofa, VC, VdC, VNF, IPCA
* Paths: AMR-Prado-IPCA, VV-Prado-IPCA, AMR-Braga-IPCA, PL-Braga-IPCA, Gmr-Braga-IPCA, Joane-VNF-IPCA, Trofa-VNF-IPCA, VC-PV-IPCA
* Schedules, trip to Campus: 9h, 11h, 14h
* Schedules, trip from Campus: 13h, 16h, 18h

This demo just takes into account a single branch and the trips to IPCA:
* Paths (branch of Prado): AMR-Prado-IPCA, VV-Prado-IPCA
* Schedules, trip to Campus: 9h, 11h, 14h

New functions created for this CSP:
* atmost_five  - returns TRUE if each vehicle is assigned to 5 or less passengers, including the driver;
* distinct_val - returns the number of distinct values (vehicles) assigned to a set of variables (persons);
* driver_on - retorns TRUE if the driver is assigned to his/her own vehicle.

----

In [7]:
from csp import *
# from notebook import psource, plot_NQueens

# %matplotlib inline
# Hide warnings in the matplotlib sections

import math
import warnings
warnings.filterwarnings("ignore")

# GET A RIDE TO AND FROM CAMPUS

# Context variables used in the implementation:
variaveis = 'P1 P2 P3 P4 P5 P6 P7 P8 P9 P10 P11'.split()

# Path: AMR-Prado-IPCA, VV-Prado-IPCA
amr_p = set(range(1, 4))
amr_v = {1, 2}                        # passengers 1-4 are from Amares and 1-2 have their own vehicle
vvd_p = set(range(4, 7))
vvd_v = {5, 6}
prd_p = set(range(7, 12))
prd_v = {10, 11} | amr_v | vvd_v     # passengers from Prado can take a ride in Amr and VV vehicles
veiculos = amr_v | vvd_v | prd_v

# Schedules
sch9  = {1, 3, 8, 10}
sch11 = {2, 4, 5} | sch9
sch14 = {6, 7, 9, 11} | sch11

# Minimum number of vehicles
mnv = math.ceil(len (amr_p | vvd_p | prd_p) / 5.0)

# domain definition
dominio =  {'P1': amr_v & sch9, 'P2': amr_v & sch11, 'P3': amr_v & sch9,
            'P4': vvd_v & sch11, 'P5': vvd_v& sch11, 'P6': vvd_v & sch14,
            'P7': prd_v & sch14, 'P8': prd_v & sch9, 'P9': prd_v & sch14, 'P10': prd_v & sch9, 'P11': prd_v & sch14
            }

# New constraint functions
def atmost_five(*values):
    """Returns True if all values are assigned to atmost 5 variables, False otherwise"""
    for v in set(values):
        if values.count(v) > 5:
            return False
    return True

def driver_on(*values):
    """Returns True if all cars include the driver, False otherwise"""
    for v in set(values):
        if values[v-1] != v:
            return False
    return True

def distinct_val(n):
    """Returns a function that is True when the the count of distinct values is <= n, False otherwise"""

    def distval(*values):
        return len(set(values)) <= n

    distval.__name__ = "distval(" + str(n) +')'
    return distval

# constraints definition
restricoes =   [
                Constraint(variaveis, distinct_val(mnv)),
                Constraint(variaveis, atmost_five),
                Constraint(variaveis, driver_on)
                ]

# Get_ride
get_ride = NaryCSP( dominio, restricoes)

# Result
# ac_solver(get_ride, arc_heuristic=sat_up)
sol_gr = ac_search_solver(get_ride, arc_heuristic=sat_up)

# print result
for v in set(sol_gr.values()):
    print('Veículo ', v, end=': ')
    for (var, val) in sol_gr.items():
        if val == v:
            print(var, end=' ')
    print()

Veículo  1: P1 P3 P8 P10 
Veículo  2: P2 P11 
Veículo  5: P4 P5 P6 P7 P9 
